In [ ]:
conda install -c conda-forge imbalanced-learn

In [ ]:
!pip install BorutaShap

In [ ]:
conda install -c conda-forge boruta_py

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from imblearn.over_sampling import SMOTE
from BorutaShap import BorutaShap, load_data
from boruta import BorutaPy
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC

In [ ]:
#set display max column to see all column
pd.set_option("display.max_columns", None)
pd.get_option("display.max_columns")

#set display max column to see all column
pd.set_option("display.max_rows", 150)
pd.get_option("display.max_rows")

In [ ]:
app_train = pd.read_csv(r'C:\Users\Admin\Documents\GitHub\CreditRiskML\data/german.csv')

In [ ]:
feature_names =    ['Creditability', 'Account_Balance', 'Duration_of_Credit_monthly', 'Payment_Status_of_Previous_Credit', 'Purpose	Credit_Amount', 'Value_Savings_Stocks', 'Length_of_current_employment', 'Instalment_per_cent', 'Sex_Marital_Status', 'Guarantors', 'Duration_in_Current_address', 'Most_valuable_available_asset', 'Age_years', 'Concurrent_Credits', 'Type_of_apartment', 'No_of_Credits_at_this_Bank', 'Occupation', 'No_of_dependents', 'Telephone', 'Foreign_Worker']

In [ ]:
app_train.tail()

In [ ]:
app_train["Payment_Status_of_Previous_Credit"].value_counts()


In [ ]:
app_train['Payment_Status_of_Previous_Credit'].astype(int).plot.hist();

In [ ]:
print(app_train.dtypes)

In [ ]:
feature_names =    ['Creditability','Account_Balance', 'Duration_of_Credit_monthly', 'Payment_Status_of_Previous_Credit',  'Credit_Amount', 'Value_Savings_Stocks',  'Length_of_current_employment',
   'Most_valuable_available_asset', 'Age_years', 'No_of_Credits_at_this_Bank', 'Occupation', 'Foreign_Worker']

In [ ]:
def yfromX(X):
    y = X['Creditability'] + X['Duration_of_Credit_monthly']**2 + np.sin(3 * X['Account_Balance']) + (X['Payment_Status_of_Previous_Credit'] * X['Credit_Amount'] * X['Age_years'])
    return y

In [ ]:
##Data that is important
#importantstuff = list(X.columns[boruta.support_])
#print(f"stuff that is important: {importantstuff}")
##Data that is important but not confirmed
#maybeimportant = list(X.columns[boruta.support_weak_])
#print(f"stuff that might be important: {maybeimportant}")
##Just garbage...
#uselessstuff = list(X.columns[~(boruta.support_ | boruta.support_weak_)])
#print(f"stuff that is just useless: {uselessstuff}")

In [ ]:
np.random.seed(0)

X = pd.DataFrame(np.random.normal(size = (20000, len(feature_names))), columns = feature_names)
y = yfromX(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [ ]:
#need to chose estimator obv
estimator = RandomForestRegressor()

In [ ]:
#object n stuff
boruta = BorutaPy(estimator = estimator, n_estimators = 'auto', max_iter = 100)
boruta.fit(np.array(X_train), np.array(y_train))

In [ ]:
##Data that is important
importantstuff = list(X.columns[boruta.support_])
print(f"stuff that is important: {importantstuff}")
##Data that is important but not confirmed
maybeimportant = list(X.columns[boruta.support_weak_])
print(f"stuff that might be important: {maybeimportant}")
##Just garbage...
uselessstuff = list(X.columns[~(boruta.support_ | boruta.support_weak_)])
print(f"stuff that is just useless: {uselessstuff}")